In [5]:
!pip install nltk
!pip install tflearn

You should consider upgrading via the 'c:\users\ammuuu\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.



  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127304 sha256=8d6df54d2d300cf9c546bd420e96f8a0d6d6ebd3d44a33dfb7d30a22238506a6
  Stored in directory: c:\users\ammuuu\appdata\local\pip\cache\wheels\5f\14\2e\1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


You should consider upgrading via the 'c:\users\ammuuu\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [4]:
stemmer = LancasterStemmer()

In [6]:
import numpy
import tflearn
import tensorflow
import random
import json

c:\users\ammuuu\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [7]:
with open("intents.json") as intents:
    data = json.load(intents)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies ar

In [8]:
data['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is tim',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I am 18 years old!', '18 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Tim.',
   "I'm Tim!",
   "I'm Tim aka Tech With Tim."],
  'context_set': ''},
 {'tag': 'shop',
  'patterns': ['Id like to buy something',
   'whats on the menu',
   'what do you reccommend?',
   'could i get som

In [13]:
#all words
words = []
#the possible labels
labels = []
#will hold the patterns
docs_x = []
#will hold the label for each pattern
docs_y = []

#loop through the intent dictionary
for intent in data['intents']:
    #loop through each pattern in each patterns list
    for pattern in intent['patterns']:
        #tokenize basically separates each word in a pattern and puts the words into a list
        wrds = nltk.word_tokenize(pattern)
        print(wrds)

['Hi']
['How', 'are', 'you']
['Is', 'anyone', 'there', '?']
['Hello']
['Good', 'day']
['Whats', 'up']
['cya']
['See', 'you', 'later']
['Goodbye']
['I', 'am', 'Leaving']
['Have', 'a', 'Good', 'day']
['how', 'old']
['how', 'old', 'is', 'tim']
['what', 'is', 'your', 'age']
['how', 'old', 'are', 'you']
['age', '?']
['what', 'is', 'your', 'name']
['what', 'should', 'I', 'call', 'you']
['whats', 'your', 'name', '?']
['Id', 'like', 'to', 'buy', 'something']
['whats', 'on', 'the', 'menu']
['what', 'do', 'you', 'reccommend', '?']
['could', 'i', 'get', 'something', 'to', 'eat']
['when', 'are', 'you', 'guys', 'open']
['what', 'are', 'your', 'hours']
['hours', 'of', 'operation']


In [27]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        #add all the wrds into words
        words.extend(wrds)
        docs_x.append(pattern)
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
len(words)

430

In [25]:
#removes the ends of the words
words = [stemmer.stem(w.lower()) for w in words]
len(words)

258

In [28]:
#remove duplicates
words = sorted(list(set(words)))
len(words)

75

In [31]:
labels = sorted(labels)

In [38]:
#One hot encoding
#the mapping is [the, she, he, him, they, was, a, guy, person]
#one-hot representation -> [0, 0, 1, 0, 0, 1, 1, 0, 1]
#he was a person
#this is a bag of words

training = []
output = []

#basically what we are doing here is one-hot encoding the data
out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] * 1
    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = numpy.array(output)
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])